参考：https://www.salesanalytics.co.jp/datascience/datascience163/

# ライブラリのインポート

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml

# カスタム変換器のための道具
from sklearn.base import BaseEstimator, TransformerMixin

# BoxCox変換器
from sklearn.preprocessing import PowerTransformer

# パイプライン構築のための道具
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb

# データの読み込み


In [4]:
# タイタニックのデータセット
dataset = fetch_openml(data_id=40945, parser="auto")
df = dataset["frame"]

df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


# カスタム変換器の作成

- 作成例1：変数選択
- 作成例2：欠損値処理
- 作成例3：Box-Cox変換 

## 作成例1：変数選択

In [24]:
class ColumnFilterTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, columns=[]):
    self.columns = columns
  
  def fit(self, X, y=None):
    return self
  
  def transform(self, X):
    return X[self.columns]

In [25]:
# 選択した変数
columns_to_keep = ["age", "fare", "sibsp", "parch"]

# インスタンスの生成
transformer = ColumnFilterTransformer(columns=columns_to_keep)

# 変換器の適用
filterd_df = transformer.transform(df)

filterd_df.head()

,age,fare,sibsp,parch
0,29.0000,211.3375,0,0
1,0.9167,151.5500,1,2
2,2.0000,151.5500,1,2
3,30.0000,151.5500,1,2
4,25.0000,151.5500,1,2


In [26]:
# 欠損値の数
filterd_df.isnull().sum()

age      263
fare       1
sibsp      0
parch      0
dtype: int64

## 作成例2：欠損値の処理

In [12]:
class CustomMedianImputer(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.medians_ = None
  
  def fit(self, X, y=None):
    # X is expected to be a DataFrame
    self.medians_ = X.median()
    return self
  
  def transform(self, X, y=None):
    # X is expected to be a DataFrame
    X_copy = X.copy()
    for column in X_copy.columns:
      X_copy[column].fillna(self.medians_[column], inplace=True)
    return X_copy

In [13]:
# インスタンスの生成
imputer = CustomMedianImputer()

# 変換器の学習
imputer.fit(filterd_df)

# 変換器の適用
imputed_data = imputer.transform(filterd_df)

imputed_data.head()

,age,fare,sibsp,parch
0,29.0000,211.3375,0,0
1,0.9167,151.5500,1,2
2,2.0000,151.5500,1,2
3,30.0000,151.5500,1,2
4,25.0000,151.5500,1,2


In [14]:
# 欠損値の買うz
imputed_data.isnull().sum()

age      0
fare     0
sibsp    0
parch    0
dtype: int64

## 作成例3：Box-Cox変換

### Box_Cox変換とは  
データの変換を行う際にべき乘変換をすることがある。  
有名なところでは、対数変換(log)を一般化したBox-Cox変換がある。  
Box-Cox変換は、正規分布から逸脱した分布を持つデータを、より正規分布に近づけるために使用される。  
Pythonでは、PowerTransformerクラスを利用する。

### カスタム変換器の作成

In [17]:
class CustomBoxCoxTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    self._estimators = {}
  
  def fit(self, X, y=None):
    X_copy  = X.copy()
    for column in X_copy.columns:
      X_copy[column]+= 1
      estimator = PowerTransformer()
      self._estimators[column] = estimator.fit(np.array(X_copy[column]).reshape(-1, 1))
    return self
  
  def transform(self, X):
    X_copy = X.copy()
    for column in X_copy.columns:
      X_copy[column] += 1
      X_copy[column] = self._estimators[column].transform(np.array(X_copy[column]).reshape(-1, 1))
    return X_copy
  
  def inverse_transform(self, X):
    X_copy = X.copy()
    for column in X_copy.columns:
      X_copy[column]= self._estimators(column).inverse_transform(np.array(X_copy[column]).reshape(-1, 1))

In [18]:
# インスタンスの生成
boxcox_trans = CustomBoxCoxTransformer()

# 変換器の学習
boxcox_trans.fit(imputed_data)

# 変換器の適用
transformed_data = boxcox_trans.transform(imputed_data)

transformed_data.head()

,age,fare,sibsp,parch
0,0.012524,2.106427,-0.681878,-0.553158
1,-2.583499,1.893590,1.361687,1.884514
2,-2.444805,1.893590,1.361687,1.884514
3,0.089024,1.893590,1.361687,1.884514
4,-0.299485,1.893590,1.361687,1.884514


# 推定器と連携しパイプラインを学習

## データの準備

In [19]:
# 目的変数yと説明変数X
y = df["survived"].astype(int) 
X = df.drop("survived", axis=1)

In [20]:
X_train,X_test, y_train, y_test = train_test_split(
  X,
  y,
  test_size=0.3,
  random_state=123
)

## パイプラインの定義

In [27]:
# 選択した変数
columns_to_keep = ["age", "fare", "sibsp", "parch"]

# パイプラインの定義
titanic_pipeline = Pipeline(
  steps=[
    ("filter", ColumnFilterTransformer(columns_to_keep)),
    ("impuer", CustomMedianImputer()),
    ("boxcoxtrans", CustomBoxCoxTransformer()),
    ("estimater", xgb.XGBClassifier())
  ]
)

## 学習とテスト

In [28]:
titanic_pipeline.fit(X_train, y_train)

Pipeline(steps=[('filter',
                 ColumnFilterTransformer(columns=['age', 'fare', 'sibsp',
                                                  'parch'])),
                ('impuer', CustomMedianImputer()),
                ('boxcoxtrans', CustomBoxCoxTransformer()),
                ('estimater',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [29]:
# 目的変数yの予測
pred_y = titanic_pipeline.predict(X_test)

# 正答率
accuracy_score(y_test, pred_y)

0.6997455470737913

## パイプラインのパラメータを変更

In [31]:
# 選択した変数
columns_to_keep = ["age", "fare"]

# パイプラインにパラメータを設定
params = {"filter__columns": columns_to_keep}
titanic_pipeline.set_params(**params)

# パイプラインの学習
titanic_pipeline.fit(X_train, y_train)

Pipeline(steps=[('filter', ColumnFilterTransformer(columns=['age', 'fare'])),
                ('impuer', CustomMedianImputer()),
                ('boxcoxtrans', CustomBoxCoxTransformer()),
                ('estimater',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metri...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [32]:
# 目的変数yの予測
pred_y = titanic_pipeline.predict(X_test)
# 正答率
accuracy_score(y_test, pred_y)

0.6895674300254453